In [106]:
# from google.colab import drive
# drive.mount('/content/drive')

In [107]:
%cd /content/drive/MyDrive/LumiNet_Files/LumiNet-lee/

/content/drive/MyDrive/LumiNet_Files/LumiNet-lee


In [108]:
!pip install -r requirements.txt

In [109]:
# ==========================================
# [Cell 1] 라이브러리 임포트 및 모델 엔진 설정 (실행 1회)
# ==========================================
import sys
import os
import torch
import cv2
import numpy as np
from PIL import Image
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
from cldm.model import load_state_dict
from cldm.ddim_hacked import DDIMSampler
import einops

# 경로 설정 (Colab 경로에 맞게 수정 필요)
BASE_MODEL_PATH = "./crossattn_checkpoints/best_crossattn_offwhite.ckpt"
CONFIG_PATH = "./models/cldm_v21_LumiNet.yaml"

# 전역 변수 (모델을 메모리에 계속 유지하기 위함)
global_model = None
global_sampler = None
current_adapter_color = None

def initialize_engine():
    """베이스 모델을 메모리에 로드합니다."""
    global global_model, global_sampler

    if global_model is not None:
        print("✅ 모델이 이미 로드되어 있습니다.")
        return

    print("🚀 베이스 모델 로딩 시작... (최초 1회)")
    config = OmegaConf.load(CONFIG_PATH)
    model = instantiate_from_config(config.model).cpu()
    model.add_new_layers()

    if os.path.exists(BASE_MODEL_PATH):
        model.load_state_dict(load_state_dict(BASE_MODEL_PATH, location='cpu'), strict=False)
        print(f"📦 Base Model 로드 완료: {BASE_MODEL_PATH}")
    else:
        raise FileNotFoundError(f"Base Model 없음: {BASE_MODEL_PATH}")


    new_decoder = True #
    if new_decoder: #
        ae_checkpoint = "./ckpt/new_decoder.ckpt" #
        model.change_first_stage(ae_checkpoint) #

    model.cuda()
    model.eval()

    global_model = model
    global_sampler = DDIMSampler(model)
    print("✨ 엔진 준비 완료!")

def switch_adapter(target_color):
    """요청된 색상으로 어댑터 가중치만 교체합니다."""
    global global_model, current_adapter_color

    if current_adapter_color == target_color:
        print(f"ℹ️ 이미 {target_color} 어댑터가 적용되어 있습니다.")
        return

    weights_path = f"./adaptors/adaptor_{target_color}.pth"

    if not os.path.exists(weights_path):
        print(f"(White 모드로 진행)")
        current_adapter_color = "none"
        return

    print(f"🔄 어댑터 교체 중... ({current_adapter_color} -> {target_color})")
    checkpoint = torch.load(weights_path, map_location='cpu')

    # Hot-Swapping (모델을 끄지 않고 가중치만 덮어쓰기)
    if 'light_encoder' in checkpoint:
        global_model.control_model.prior_extracter.model_latents.light_encoder.load_state_dict(checkpoint['light_encoder'])
    if 'light_decoder' in checkpoint:
        global_model.control_model.prior_extracter.light_decoder.load_state_dict(checkpoint['light_decoder'])

    current_adapter_color = target_color
    print(f"✅ {target_color} 어댑터 장착 완료!")

def run_process(inference_root, color, guidance_scale=9.0, ddim_steps=50):
    """실제 추론을 수행하는 함수"""
    # 1. 어댑터 교체 시도
    switch_adapter(color)
    new_decoder = True #

    print(f"\n📂 처리 시작: {inference_root} (Color: {color})")

    for root, dirs, files in os.walk(inference_root):
        off_name = next((f for f in files if 'off' in f.lower()), None)

        if off_name:
            off_path = os.path.join(root, off_name)
            save_path = os.path.join(root, f"output_{color}.jpg")
            print(f"▶ Processing: {os.path.basename(root)}")

            # 이미지 로드 & 전처리
            img_off = Image.open(off_path).convert("RGB")
            orig_w, orig_h = img_off.size
            img_off_resized = img_off.resize((512, 512), Image.BICUBIC)
            img_ref_resized = Image.new("RGB", (512, 512), (255, 255, 255))

            t_off = torch.from_numpy(np.array(img_off_resized).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0).cuda()
            t_ref = torch.from_numpy(np.array(img_ref_resized).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0).cuda()
            hint = torch.cat((t_off, t_ref), dim=1)


            input_image_full = cv2.imread(off_path)[..., ::-1] / 255.0 #
            inp = cv2.resize(input_image_full, (512, 512)) #
            input_img = torch.from_numpy(inp.copy()).float().cuda() #
            input_img = input_img.unsqueeze(0) #
            input_img = einops.rearrange(input_img, 'b h w c -> b c h w').clone() #

            # 추론
            with torch.no_grad():
                c_cat = hint
                c = global_model.get_unconditional_conditioning(c_cat.shape[0])
                cond = {"c_concat": [c_cat], "c_crossattn": [c]}
                shape = (4, 512 // 8, 512 // 8)

                samples, _ = global_sampler.sample(ddim_steps, 1, shape, cond, verbose=False,
                                            unconditional_guidance_scale=guidance_scale)


                if new_decoder: #
                    ae_hs = global_model.encode_first_stage(input_img * 2 - 1)[1] #
                    x_sample = global_model.decode_new_first_stage(samples, ae_hs) #
                else: #
                    x_sample = global_model.decode_first_stage(samples) #
                x_sample = torch.clamp((x_sample + 1.0) / 2.0, min=0.0, max=1.0) #

                result_numpy = x_sample.cpu().permute(0, 2, 3, 1).numpy()[0] * 255
                result_numpy = result_numpy.astype(np.uint8)
                result_final = cv2.resize(result_numpy, (orig_w, orig_h), interpolation=cv2.INTER_LANCZOS4)

                cv2.imwrite(save_path, cv2.cvtColor(result_final, cv2.COLOR_RGB2BGR))
                print(f"  💾 Saved: {save_path}")

# 초기화 실행
initialize_engine()

🚀 베이스 모델 로딩 시작... (최초 1회)
ControlLDM: Running in v-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAtten

In [110]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'white'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

(White 모드로 진행)

📂 처리 시작: ./images/42 (Color: white)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.56it/s]


  💾 Saved: ./images/42/output_white.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.77it/s]


  💾 Saved: ./images/42/42/output_white.jpg


In [111]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'red'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (none -> red)
✅ red 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: red)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.78it/s]


  💾 Saved: ./images/42/output_red.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.81it/s]


  💾 Saved: ./images/42/42/output_red.jpg


In [112]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'orange'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (red -> orange)
✅ orange 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: orange)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.78it/s]


  💾 Saved: ./images/42/output_orange.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.70it/s]


  💾 Saved: ./images/42/42/output_orange.jpg


In [113]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'yellow'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (orange -> yellow)
✅ yellow 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: yellow)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.72it/s]


  💾 Saved: ./images/42/output_yellow.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.89it/s]


  💾 Saved: ./images/42/42/output_yellow.jpg


In [114]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'green'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (yellow -> green)
✅ green 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: green)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.71it/s]


  💾 Saved: ./images/42/output_green.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.82it/s]


  💾 Saved: ./images/42/42/output_green.jpg


In [115]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'blue'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (green -> blue)
✅ blue 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: blue)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.80it/s]


  💾 Saved: ./images/42/output_blue.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.74it/s]


  💾 Saved: ./images/42/42/output_blue.jpg


In [116]:
# ==========================================
# [Cell 2] 추론 실행 (반복 실행 가능)
# ==========================================

# 1. 원하는 색상과 설정 입력
TARGET_COLOR = 'purple'  # red, green, blue 등 변경
INFERENCE_PATH = "./images/42"

# 2. 실행 함수 호출
run_process(
    inference_root=INFERENCE_PATH,
    color=TARGET_COLOR,
    guidance_scale=9.0
)

🔄 어댑터 교체 중... (blue -> purple)
✅ purple 어댑터 장착 완료!

📂 처리 시작: ./images/42 (Color: purple)
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.81it/s]


  💾 Saved: ./images/42/output_purple.jpg
▶ Processing: 42
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.79it/s]


  💾 Saved: ./images/42/42/output_purple.jpg


In [117]:
# !cp -r "/content/drive/MyDrive/LumiNet_Files/LumiNet-choi/modi_vae" "/content/drive/MyDrive/LumiNet_Files/LumiNet-lee/modi_vae"


In [118]:
# !cp "/content/drive/MyDrive/LumiNet_Files/LumiNet-choi/Load_model.py" "/content/drive/MyDrive/LumiNet_Files/LumiNet-lee/"